In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

In [4]:
messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

llm.invoke(messages)

AIMessage(content='어머, 개스톤! 안녕하세요.\n\n음... 죄송하지만, 오늘은 좀 어려울 것 같아요. 저는 지금 읽고 있던 책을 마저 읽어야 하고, 또 다른 할 일들도 많아서요.\n\n저녁은 괜찮습니다. 하지만 제안해주셔서 고마워요.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--16431ce0-9140-4ab8-8cfd-740932f3ac70-0', usage_metadata={'input_tokens': 45, 'output_tokens': 214, 'total_tokens': 259, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 152}})

### StrOutputParser

+ 텍스트 결과만 필요한 경우 사용하는 출력 파서(output parser)
+ 출력 파서는 언어 모델이 반환하는 결과에서 원하는 형식의 데이터를 추출하는 도구

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = llm.invoke(messages)
parser.invoke(result)

'아, 개스톤이시군요. 음... 초대해주셔서 감사하지만, 저는 오늘 저녁에 읽어야 할 책이 너무 많아요.\n\n도서관에서 빌려온 책들이 저를 애타게 기다리고 있답니다. 책 속 세상은 언제나 저를 더 흥미롭게 만들거든요.\n\n다음에 뵐게요. 저는 이만 책 속으로 들어가 봐야겠어요.'

### LCEL(LangChain Expression Language)

+ LangChain에서 구성 요소를 쉽게 조합하고 사용자 정의 체인(Chain)을 만들 수 있도록 지원하는 기능
+ 모든 구성 요소를 연결하여 복잡한 워크플로우를 구축
+ 유연성, 확장성, 디버깅 용이성 등의 장점

In [8]:
chain = llm | parser
chain.invoke(messages)

'오, 개스톤! 예상치 못한 만남이네요. 저녁 말씀이신가요?\n\n음... 죄송하지만, 오늘은 좀 어려울 것 같아요. 마침 도서관에서 빌려온 책의 가장 흥미로운 부분에 다다랐거든요. 저녁 내내 그 책과 함께 시간을 보내고 싶어요.'

### 프롬프트 템플릿

**프롬프트 템플릿(Prompt Template)이란?**

+ 언어 모델(LLM)에 전달될 프롬프트의 구조와 내용을 정의하는 서식
+ placeholder 사용(런타임 중 동적으로 값 삽입)
+ 정형화된 입력, LLM의 일관된 응답 유도
+ 프롬프트를 효율적으로 관리

**주요 기능 및 특징**

+ 변수 관리: input_variables를 통해 프롬프트 내에서 사용될 변수들을 명시적으로 정의한다.
+ 동적 생성: format() 메서드를 사용하여 변수에 실제 값을 채워 넣어 완성된 프롬프트 문자열을 생성한다.
+ 재사용성: 한 번 정의된 템플릿은 다양한 입력에 대해 재사용할 수 있어 코드의 중복을 줄인다.
+ 가독성 및 유지보수성: 프롬프트의 구조가 명확해져 가독성이 높아지고, 변경 사항 발생 시 유지보수가 용이하다.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕하세요? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


In [10]:
chain = prompt_template | llm | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'오, 야수님. 저에게 저녁 식사를 제안하시다니... 조금 놀랍네요.\n\n하지만 괜찮습니다. 함께 시간을 보내는 것도 나쁘지 않겠군요. 어떤 저녁이 될지는 모르겠지만요. 초대해주셔서 고마워요.'

### JsonOutputParser

+ JSON 형식의 문자열 출력을 파싱하여 Python 딕셔너리 또는 리스트로 변환
+ LLM이 JSON 형식으로 응답을 생성하도록 지시할 때 유용
+ 파싱 오류 발생 시 적절한 예외 처리 필요


In [13]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# JSON으로 출력될 형식 정의
parser = JsonOutputParser()

# LLM이 특정 형식의 JSON을 생성하도록 지시하는 프롬프트
prompt = PromptTemplate(
    template="{query}.\n{format_instructions}",
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

# 체인 구성
chain = prompt | llm | parser

# 실행 및 결과 출력
result = chain.invoke({"query": "미녀와 야수에 나오는 등장인물 세 명과 각 등장인물의 특징을 JSON 형태로 소개해줘"})
print(result)


{'characters': [{'name': '벨 (Belle)', 'characteristics': ['독서를 좋아하고 똑똑하며 독립적인 성격입니다.', '겉모습보다 내면의 아름다움을 중요하게 생각합니다.', '마을 사람들과는 다른 자신만의 가치관을 가지고 있습니다.']}, {'name': '야수 (Beast)', 'characteristics': ['마법에 걸려 야수의 모습으로 변한 왕자입니다.', '처음에는 거칠고 오만하지만, 벨을 통해 점차 친절하고 따뜻한 마음을 배우게 됩니다.', '외로움을 느끼며 진정한 사랑을 갈망합니다.']}, {'name': '개스톤 (Gaston)', 'characteristics': ['마을에서 가장 잘생기고 힘센 사냥꾼으로, 자기애가 강하고 오만합니다.', '벨에게 집착하며, 자신의 뜻대로 되지 않자 폭력적이고 비열한 모습을 보입니다.', '겉모습만 중시하며 내면의 가치를 이해하지 못합니다.']}]}
